# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [52]:
import os
import sys
import random
import math
import numpy as np
import skimage
import matplotlib
import matplotlib.pyplot as plt

import coco
import utils
import model as modellib
import visualize

%matplotlib inline 

%load_ext autoreload
%autoreload 2

# Root directory of the project
ROOT_DIR = os.getcwd()
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os, sys
os.getcwd()

'/home/jeremy/Mask_RCNN'

In [145]:
import cv2
import imutils
VID_DIR = os.path.join(IMAGE_DIR, "cycling")

n_rides = 4
rotate = [True,False,True,True]
limited = True
sample_rate = 1 # calculates every sample_rate image
max_images = 50 # limit of total frames to generate per video

for n in range(n_rides):
    num_vid = (str(n)).zfill(3)
    FRAME_DIR = IMAGE_DIR + '/cycling/ride_'+ num_vid + '/frames/'
    old_frames = os.listdir(FRAME_DIR)
    
    for f in old_frames:
        os.remove(FRAME_DIR+f)

    
    vidcap = cv2.VideoCapture(VID_DIR+'/ride_' + num_vid + '/vid/ride_' + num_vid + '.mp4' )
    success,image = vidcap.read()
    count = 0
    success = True
    print('processing: ' +VID_DIR+'/ride_' + num_vid + '/vid/ride_' + num_vid + '.mp4' )
    while success:
        # TODO limit doesnt work, inefficient slow way of loadng videos
        if count % sample_rate == 0:
            if rotate[n]:
                image = imutils.rotate(image, -90)

            cv2.imwrite(VID_DIR+ '/ride_' + num_vid + '/frames/%2.3d.png' % count, image) # save frame as .png file      
            success,image = vidcap.read()
        count += 1
        if count > max_images:
            success = False
    
print ('Created all images from videos succesfully.')
        

processing: /home/jeremy/Mask_RCNN/images/cycling/ride_000/vid/ride_000.mp4
processing: /home/jeremy/Mask_RCNN/images/cycling/ride_001/vid/ride_001.mp4
processing: /home/jeremy/Mask_RCNN/images/cycling/ride_002/vid/ride_002.mp4
processing: /home/jeremy/Mask_RCNN/images/cycling/ride_003/vid/ride_003.mp4
Created all images from videos succesfully.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [42]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINING     

## Create Model and Load Trained Weights

In [44]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

/home/jeremy/Mask_RCNN/venv_cycling/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [45]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [218]:
# Remove previous predictions


# Save each output image prediction
limit = 50
for ride in [3]:#range(n_rides):
    
    
    FRAME_DIR = IMAGE_DIR + '/cycling/ride_'+ str(ride).zfill(3) + '/frames/'
    
    PRED_DIR = IMAGE_DIR + '/cycling/ride_' + str(ride).zfill(3) + '/results/'
    
    old_preds = os.listdir(PRED_DIR)
    
    for f in old_preds:
        os.remove(PRED_DIR+f)
        
    file_names = os.listdir(FRAME_DIR)#next(os.walk(FRAME_DIR))[2]
    file_names.sort()
    for count, f in enumerate(file_names):
        
        if count > limit:
            break
        else:
            image = skimage.io.imread(os.path.join(FRAME_DIR, f))

            # Run detection
            results = model.detect([image], verbose=0)

            # Visualize results
            r = results[0]
            
            idx = np.where(r['class_ids'] == 1)[0]
#             r = r[idx[0]]
            visualize.display_instances(image, r['rois'][idx], r['masks'][:,:,idx], r['class_ids'][idx], 

                                        class_names, r['scores'][idx], display_box=False,preview=False,
                                        save_dir= PRED_DIR + f)